# Problem 3A

In [216]:
means = df.mean()
print(means)

# center data
center_df = df - means
# print(center_df)

a    0.008677
a    0.008777
a    0.009921
a    0.007468
a    0.009568
dtype: float64


In [215]:
# import csv
import datetime
import pandas as pd
import numpy as np

def read_valueshist(filename):
    "Return a dictionary mapping dates to values"
    with open(filename) as f:
        reader = csv.DictReader(f)
        mydict = {}
        for row in reader:
            mydict[datetime.datetime.strptime(row['Date'], '%m/%d/%Y')] = row['Value_With_Dividends__USD_']
    return mydict

def next_month(d):
    if d.month == 12:
        return datetime.datetime(d.year + 1, 1, 1)
    else:
        return datetime.datetime(d.year, d.month+1, 1)


start = datetime.datetime(1996, 7, 1)
end = datetime.datetime(2023, 12, 1)
months = (end.year - start.year) * 12 + end.month - start.month
indices = [ read_valueshist(f)
            for f in ["valueshist_D41440.csv", "valueshist_D41443.csv", "valueshist_D41446.csv",
                     "valueshist_D41455.csv", "valueshist_D41458.csv"]]

files = ["valueshist_D41440.csv", "valueshist_D41443.csv", "valueshist_D41446.csv",
                     "valueshist_D41455.csv", "valueshist_D41458.csv"]

dfs = []
for f in files:
    mydict = dict(sorted(read_valueshist(f).items()))
    keys = sorted(mydict.keys())
    date_range = [start + datetime.timedelta(days=x) for x in range((end - start).days + 1)]
    missing_dates = [date for date in date_range if date not in mydict]
    for date in missing_dates:
        next_date = max([k for k in keys if k < date], default=None)
        mydict[date] = mydict[next_date]
    sort_dict = dict(sorted(mydict.items()))
    df_raw = pd.DataFrame(sort_dict.items(), columns=["date", "value"])
    df_filter = df_raw[(df_raw["date"].dt.day == 1) & (df_raw["date"] >= start) & (df_raw["date"] <= end)].apply(pd.to_numeric)
    df_filter["a"] = df_filter["value"] / df_filter["value"].shift(1) - 1
    dfs.append(df_filter["a"])

df = pd.concat(dfs, axis=1)
df = df.iloc[1:]

# print(len(df))
print(df)


              a         a         a         a         a
31    -0.014936 -0.030453 -0.037764 -0.022209 -0.059043
62     0.018667  0.000902  0.009837 -0.005172 -0.005350
92     0.021804  0.031438  0.044329  0.024336  0.002920
123    0.017782  0.003513  0.018608 -0.006594 -0.011824
153    0.054398  0.049305  0.066904  0.040192 -0.001644
184    0.026502 -0.007583 -0.007091 -0.005948 -0.031062
215    0.003614  0.005862  0.040812 -0.029510  0.097808
243    0.006768  0.010487  0.011883  0.007968  0.024149
274    0.005266 -0.020019 -0.032469 -0.008815 -0.025670
304    0.019116  0.030801  0.035652  0.028263  0.016506
335    0.038040  0.063755  0.064687  0.065549  0.037401
365    0.068180  0.050088  0.049205  0.051653  0.041520
396    0.006342  0.017915  0.051498 -0.008781  0.019244
427   -0.023961 -0.045293 -0.026532 -0.056039 -0.092715
457    0.090700  0.053181  0.060521  0.048921  0.034672
488   -0.041995 -0.051728 -0.037231 -0.050658 -0.176699
518    0.015647  0.012420  0.064168 -0.021790 -0

In [217]:
import numpy as np

# compute SVD
U, S, V = np.linalg.svd(center_df, full_matrices=False)
print(U, S)

[[-0.04595826  0.03252334 -0.06722741  0.01878347 -0.05480005]
 [-0.0056941   0.01948751 -0.00843734  0.08403226  0.01181055]
 [ 0.01575869  0.04890155  0.0401786  -0.02842047 -0.0217476 ]
 ...
 [-0.0371431  -0.03689416 -0.05082215  0.00322236  0.03138129]
 [-0.04522252  0.00012646  0.01345446  0.00441366 -0.01250681]
 [ 0.07954431  0.04929313  0.01103026  0.00981357 -0.00194554]] [2.04910722 0.60696051 0.38967306 0.19085036 0.01963642]


In [193]:
# compute eta_k values

eta_k = np.cumsum(S**2) / np.sum(S**2)
print(eta_k * 100)

[ 96.65623465  99.8922909   99.97596553 100.         100.        ]
